In [1]:
import pandas as pd
import numpy as np

In [2]:
folder = 'C:/Users/jullian/Desktop/Jullian/Jullian/Data/DANS/OViN_2011-2017_Final/OViN_2011-2017_Final/'
filename = 'OVIN_2011-2017.csv'
path = folder + filename
ovin = pd.read_csv(path)

ovin['VerplNr'] = pd.to_numeric(ovin['VerplNr'],errors = 'coerce')
ovin['AankPC'] = pd.to_numeric(ovin['AankPC'],errors = 'coerce')
ovin['VertPC'] = pd.to_numeric(ovin['VertPC'],errors = 'coerce')
ovin['BuurtAdam'] = pd.to_numeric(ovin['BuurtAdam'],errors = 'coerce')


C:\Users\jullian\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3051: DtypeWarning: Columns (34,35,89,90,95,96,97,98,101,102,107,108,109,110,117,130,143,150,165,173,174,179) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#ovin['OPID'] = ovin['OPID'].astype('str')
#ovin['Jaar'] = ovin['Jaar'].astype('str')
#ovin['OPID2'] = ovin['OPID'].str.cat(ovin['Jaar'])
#print(len(ovin['OPID2']))
#print(len(ovin['OPID2'].unique()))
#print(len(ovin['OPID'].unique()))

In [3]:
# This function creates a column which tells whether the individual made a trip in Amsterdam
def Amsterdam_trip(ovin):
    #ovin['VerplNr'] = pd.to_numeric(ovin['VerplNr'],errors = 'coerce')
    # add column with postal code area of a person's home
    #ovin['WoonPC'] = np.where(((ovin['Vertrekp']==int(1)) &  (ovin['VerplNr']==int(1))), ovin['VertPC'], np.NaN)
    ovin['Amsterdam_vpl'] = np.where(((ovin['VertGem']== '363') | (ovin['AankGem']=='363')),1,np.NaN)
    # persons with same id have same home
    #print(ovin[['OPID','Amsterdam_vpl']])
    ovin['Amsterdam_vpl'] = ovin[['OPID','Amsterdam_vpl']].groupby('OPID').ffill()
    ovin['Amsterdam_vpl'] = ovin[['OPID','Amsterdam_vpl']].groupby('OPID').bfill()
    ovin['Amsterdam_vpl'] = ovin['Amsterdam_vpl'].fillna(0)
    #print(ovin[['OPID','Amsterdam_vpl']])
    return ovin

In [4]:
# This function creates 
def Car_trip(ovin):
    # add column with postal code area of a person's home
    #ovin['WoonPC'] = np.where(((ovin['Vertrekp']==int(1)) &  (ovin['VerplNr']==int(1))), ovin['VertPC'], np.NaN)
    ovin['car_trip'] = np.where((ovin['RVm']== '6'),1,np.NaN)
    # persons with same id have same home
    ovin['car_trip'] = ovin[['OPID','car_trip']].groupby('OPID').ffill()
    ovin['car_trip'] = ovin[['OPID','car_trip']].groupby('OPID').bfill()
    ovin['car_trip'] = ovin['car_trip'].fillna(0)
    #print(ovin[['OPID','car_trip']])
    return ovin

In [5]:
def thuislocatie(ovin):
    ovin['home'] = np.where(((ovin['VerplNr'] == 1) & (ovin['Vertrekp'] == 1)),ovin['VertPC'],np.NaN)
    ovin['home'] = ovin[['OPID','home']].groupby('OPID').ffill()
    ovin['home'] = ovin['home'].fillna(0)
    return ovin

In [6]:
temp = Amsterdam_trip(ovin)
temp = Car_trip(temp)
temp = thuislocatie(temp)

['__call__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__self__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__text_signature__']

In [7]:
l = temp[temp['Amsterdam_vpl']==1]
print(len(l['OPID'].unique())) # aantal individuen die een verplaatsing in Amsterdam heeft gemaakt.
l = l[l['car_trip']==1]
len(l['OPID'].unique()) #aantal individuen die met de auto een verplaatsing in Amsterdam hebben gedaan. 

15514


4966

In [76]:
Filtered_ovin = temp[((temp['Amsterdam_vpl']== 1) & (temp['car_trip']== 1))]
PC4_Amsterdam = np.where(Filtered_ovin['BuurtAdam'] != 0,Filtered_ovin['home'],0)
PC4_Amsterdam = np.unique(PC4_Amsterdam)

# count how many individuals travelled to a specific AankPC 
count_AankPC = Filtered_ovin.groupby('AankPC').OPID.nunique()[PC4_Amsterdam]
count_VertPC = Filtered_ovin.groupby('VertPC').OPID.nunique()[PC4_Amsterdam]
count_BuurtAdam = Filtered_ovin.groupby('BuurtAdam').OPID.nunique()

# Filter the counts on amsterdam PC
#count_AankPC
#count_BuurtAdam
#count_AankPC
#PC4_Amsterdam
#Filtered_ovin.groupby('AankPC').OPID.nunique()

#print(Filtered_ovin['AankPC'].value_counts())
#print(Filtered_ovin['BuurtAdam'].value_counts())

count_AankPC.to_csv(r'C:\Users\jullian\Desktop\Jullian\Jullian\Data\SimPark\Jullian\Untitled Folder\AankPC_count.csv')
count_VertPC.to_csv(r'C:\Users\jullian\Desktop\Jullian\Jullian\Data\SimPark\Jullian\Untitled Folder\VertPC_count.csv')
count_BuurtAdam.to_csv(r'C:\Users\jullian\Desktop\Jullian\Jullian\Data\SimPark\Jullian\Untitled Folder\BuurtAdam_count.csv')
#Filtered_ovin.to_csv(r'C:\Users\jullian\Desktop\Jullian\Jullian\Data\SimPark\Jullian\Untitled Folder\Filtered_ovin.csv')

C:\Users\jullian\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
C:\Users\jullian\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
C:\Users\jullian\Anaconda3\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


In [96]:
Filtered_ovin['home'].unique()

array(['1423', '1464', '1069', 0, '1034', '1013', '1032', '1107', '1962',
       '1052', '1112', '1025', '1074', '2121', '1079', '1066', '6181',
       '9291', '1016', '1062', '1326', '2151', '1067', '1077', '1104',
       '1028', '1115', '1508', '1051', '3089', '2011', '1611', '1086',
       '1171', '1335', '1541', '1035', '3454', '1561', '4005', '1021',
       '1181', '1060', '1771', '9244', '1214', '1132', '2406', '1566',
       '2036', '1443', '2134', '1943', '1187', '2012', '1059', '1462',
       '1338', '1218', '1622', '1628', '1313', '1053', '1451', '1057',
       '2132', '1411', '1401', '1328', '1141', '1087', '2318', '1095',
       '1082', '1017', '1901', '1098', '2135', '1448', '1075', '6717',
       '6997', '8224', '1078', '1033', '1442', '1521', '2451', '1363',
       '1834', '2343', '1445', '1318', '1018', '2033', '1967', '1319',
       '1991', '2271', '1355', '1504', '1103', '3221', '2742', '1058',
       '8243', '1317', '1273', '2025', '8031', '2101', '2334', '1186',
   

In [13]:
#pd.crosstab(Filtered_ovin['VertGem'],Filtered_ovin['AankGem'])
ovin[['OPID','Amsterdam_vpl']]

,OPID,Amsterdam_vpl
0,11101001,0.0
1,11101001,0.0
2,11101002,0.0
3,11101003,0.0
4,11101003,0.0
...,...,...
908380,17451072,0.0
908381,17451072,0.0
908382,17451072,0.0
908383,17451072,0.0
